In [1]:
import tkinter as tk
from tkinter import messagebox
import random
import math

class TicTacToeGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Tic-Tac-Toe with AI")
        self.root.resizable(False, False)
        
        # Game variables
        self.board = [' '] * 9
        self.human_player = 'X'
        self.ai_player = 'O'
        self.current_player = self.human_player
        self.game_over = False
        
        # Colors
        self.bg_color = "#f0f0f0"
        self.button_color = "#ffffff"
        self.text_color = "#333333"
        self.win_color = "#4CAF50"
        self.tie_color = "#FFC107"
        
        # Create GUI
        self.setup_gui()
        
        # Start the game
        self.reset_game()
    
    def setup_gui(self):
        """Set up the game interface"""
        self.root.configure(bg=self.bg_color)
        
        # Game status label
        self.status_label = tk.Label(
            self.root, 
            text="Your turn (X)", 
            font=('Helvetica', 14), 
            bg=self.bg_color,
            fg=self.text_color
        )
        self.status_label.grid(row=0, column=0, columnspan=3, pady=10)
        
        # Game board buttons
        self.buttons = []
        for i in range(9):
            row, col = divmod(i, 3)
            button = tk.Button(
                self.root, 
                text=' ', 
                font=('Helvetica', 24, 'bold'),
                width=5, 
                height=2,
                bg=self.button_color,
                fg=self.text_color,
                command=lambda idx=i: self.on_button_click(idx)
            )
            button.grid(row=row+1, column=col, padx=5, pady=5)
            self.buttons.append(button)
        
        # Control buttons
        control_frame = tk.Frame(self.root, bg=self.bg_color)
        control_frame.grid(row=4, column=0, columnspan=3, pady=10)
        
        tk.Button(
            control_frame, 
            text="New Game", 
            font=('Helvetica', 12),
            command=self.reset_game
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            control_frame, 
            text="Quit", 
            font=('Helvetica', 12),
            command=self.root.quit
        ).pack(side=tk.LEFT, padx=5)
    
    def reset_game(self):
        """Reset the game state"""
        self.board = [' '] * 9
        self.current_player = self.human_player
        self.game_over = False
        
        # Reset buttons
        for i in range(9):
            self.buttons[i].config(
                text=' ',
                state=tk.NORMAL,
                bg=self.button_color
            )
        
        self.update_status("Your turn (X)")
        
        # If AI goes first
        if self.ai_player == 'X':
            self.current_player = self.ai_player
            self.ai_move()
    
    def on_button_click(self, position):
        """Handle player moves"""
        if self.game_over or self.current_player != self.human_player:
            return
            
        if self.board[position] == ' ':
            self.make_move(position, self.human_player)
            
            if not self.game_over:
                self.current_player = self.ai_player
                self.update_status("AI is thinking...")
                self.root.after(500, self.ai_move)  # Small delay for better UX
    
    def make_move(self, position, player):
        """Make a move on the board"""
        self.board[position] = player
        self.buttons[position].config(
            text=player,
            state=tk.DISABLED,
            fg='red' if player == 'X' else 'blue'
        )
        
        winner = self.check_winner()
        if winner:
            self.handle_game_end(winner)
        elif ' ' not in self.board:
            self.handle_game_end('Tie')
    
    def ai_move(self):
        """AI makes a move using minimax algorithm"""
        if self.game_over:
            return
            
        best_score = -math.inf
        best_move = None
        
        for i in range(9):
            if self.board[i] == ' ':
                self.board[i] = self.ai_player
                score = self.minimax(self.board, 0, False)
                self.board[i] = ' '
                
                if score > best_score:
                    best_score = score
                    best_move = i
        
        if best_move is not None:
            self.make_move(best_move, self.ai_player)
            self.current_player = self.human_player
            self.update_status("Your turn (X)")
    
    def minimax(self, board, depth, is_maximizing):
        """Minimax algorithm with alpha-beta pruning"""
        result = self.check_winner()
        
        if result == self.ai_player:
            return 10 - depth
        elif result == self.human_player:
            return -10 + depth
        elif result == 'Tie':
            return 0
        
        if is_maximizing:
            best_score = -math.inf
            for i in range(9):
                if board[i] == ' ':
                    board[i] = self.ai_player
                    score = self.minimax(board, depth + 1, False)
                    board[i] = ' '
                    best_score = max(score, best_score)
            return best_score
        else:
            best_score = math.inf
            for i in range(9):
                if board[i] == ' ':
                    board[i] = self.human_player
                    score = self.minimax(board, depth + 1, True)
                    board[i] = ' '
                    best_score = min(score, best_score)
            return best_score
    
    def check_winner(self):
        """Check if there's a winner"""
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]
        
        for condition in win_conditions:
            if self.board[condition[0]] == self.board[condition[1]] == self.board[condition[2]] != ' ':
                return self.board[condition[0]]
        
        if ' ' not in self.board:
            return 'Tie'
        
        return None
    
    def handle_game_end(self, result):
        """Handle game end scenarios"""
        self.game_over = True
        
        # Highlight winning cells
        if result in ['X', 'O']:
            win_conditions = [
                [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
                [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
                [0, 4, 8], [2, 4, 6]              # diagonals
            ]
            
            for condition in win_conditions:
                if (self.board[condition[0]] == self.board[condition[1]] == self.board[condition[2]] == result):
                    for pos in condition:
                        self.buttons[pos].config(bg=self.win_color)
                    break
        
        # Disable all buttons
        for button in self.buttons:
            button.config(state=tk.DISABLED)
        
        # Show result message
        if result == 'Tie':
            for i in range(9):
                if self.board[i] == ' ':
                    self.buttons[i].config(bg=self.tie_color)
            messagebox.showinfo("Game Over", "It's a tie!")
            self.update_status("Game ended in a tie!")
        else:
            winner = "You" if result == self.human_player else "AI"
            messagebox.showinfo("Game Over", f"{winner} won!")
            self.update_status(f"{winner} won!")
    
    def update_status(self, message):
        """Update the status label"""
        self.status_label.config(text=message)

if __name__ == "__main__":
    root = tk.Tk()
    game = TicTacToeGUI(root)
    root.mainloop()